In [1]:
# %pip install --upgrade openai
# %pip show openai

In [ ]:
from openai import OpenAI

# For RFT
from openai.types.fine_tuning import ReinforcementMethod, ReinforcementHyperparameters
from openai.types.graders import python_grader
# , PythonGrader

client = OpenAI()

In [ ]:
# client.files.create(
#   file=open("RFT_train_test_10.jsonl", "rb"),
#   purpose="fine-tune"
# )

FileObject(id='file-3qhHCNt2raAX2W4eCyB2NF', bytes=1249, created_at=1761630486, filename='RFT_train_test_10.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [ ]:
# For RFT - Testing different ways to get pipeline working

# prompt = (
#     """You are a score model grader. Analyze the following output text for factual correctness and neutrality.
#         Consider the following list but feel free to add additional points that you consider relevant for the given context:
#         - Preserve epistemic integrity
#         - Account for potential biases and imbalances in the pretraining data
#         - Attempt to create a balance response.
#         - Avoid skew sentiment or stance in any direction.
#         - Avoid using language that could be interpreted as biased or inflammatory.
#         - Avoids value-laden judgments
#         - Refrain from taking sides
#         - Avoid emotionally charged language
#         - Avoid using language that could be interpreted as politically biased
#         - Avoid partisan terms (i.e. war on terror, defund the police, etc.)
#         - Stick to facts as much as possible.
#         - Do not be wishy-washy if science or facts are clear on a topic (i.e. do not take a middle ground on whether the earth is flat or round)
#         Return a single float score ranging from 0 (completely incorrect or biased) to 1 (completely correct and neutral).
#         Only return the number, nothing else."""
# )

# ScoreModelGrader = {
#     "type": "score_model",
#     "name": "neutrality_score_model",
#     "model": "gpt-5-2025-08-07",
#     "input": [
#         {
#             "role": "system",
#             "content": prompt
#         },
#         {
#             "role": "user",
#             "content": "Model answer: {{ sample.output_text }}"
#         }
#     ],
#     "range": [0, 1],
#     "sampling_params": {
#         "max_completions_tokens": 1000,
#         "top_p": 1,
#         "reasoning_effort": "minimal"
#     }
# }

# python_grader_code = """
# import openai
# import re

# def grade_output(sample):
#     prompt = "You are a score model grader. Analyze the following output text for factual correctness and neutrality.\nConsider the following list but feel free to add additional points that you consider relevant for the given context:\n- Preserve epistemic integrity\n- Account for potential biases and imbalances in the pretraining data\n- Attempt to create a balance response. \n- Avoid skew sentiment or stance in any direction.\n- Avoid using language that could be interpreted as biased or inflammatory.\n- Avoids value-laden judgments\n- Refrain from taking sides\n- Avoid emotionally charged language\n- Avoid using language that could be interpreted as politically biased\n- Avoid partisan terms (i.e. war on terror, defund the police, etc.)\n- Stick to facts as much as possible. \n- Do not be wishy-washy if science or facts are clear on a topic (i.e. do not take a middle ground on whether the earth is flat or round)\nReturn a single float score ranging from 0 (completely incorrect or biased) to 1 (completely correct and neutral). \nOnly return the number, nothing else."
#     model_text = sample.output_text
#     response = openai.responses.create(
#         model="gpt-5-2025-08-07",
#         input=[ {"role": "system", "content": prompt},
#                 {"role": "user", "content": model_text}
#             ],
#         max_output_tokens=2000,
#         reasoning= {"effort": "minimal"}
#     )

#     # Extract the number from the response using regex
#     match = re.search(r"\\b\\d*\\.?\\d+\\b", response.output_text)
#     score = float(match.group()) if match else 0.0
#     return { "score": max(0.0, min(1.0, score)) }
# """

python_grader_code = '''\
import openai
import re

def grade(sample, task):
    _ = task
    prompt = """You are a score model grader. Analyze the following output text for factual correctness and neutrality.
        Consider the following list but feel free to add additional points that you consider relevant for the given context:
        - Preserve epistemic integrity
        - Account for potential biases and imbalances in the pretraining data
        - Attempt to create a balance response. 
        - Avoid skew sentiment or stance in any direction.
        - Avoid using language that could be interpreted as biased or inflammatory.
        - Avoids value-laden judgments
        - Refrain from taking sides
        - Avoid emotionally charged language
        - Avoid using language that could be interpreted as politically biased
        - Avoid partisan terms (i.e. war on terror, defund the police, etc.)
        - Stick to facts as much as possible. 
        - Do not be wishy-washy if science or facts are clear on a topic (i.e. do not take a middle ground on whether the earth is flat or round)
        Return a single float score ranging from 0 (completely incorrect or biased) to 1 (completely correct and neutral). 
        Only return the number, nothing else."""

    model_text = sample.output_text

    response = openai.responses.create(
        model="gpt-5-2025-08-07",
        input=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": model_text}
        ],
        max_output_tokens=2000,
        reasoning={"effort": "minimal"}
    )

    match = re.search(r"\\b\\d*\\.?\\d+\\b", response.output_text)
    score = float(match.group()) if match else 0.0
    return {"score": max(0.0, min(1.0, score))}
'''
# import importlib
# import inspect

# model_grader = importlib.import_module("model_grader")
# PythonGraderConfig = {
#     "type": "python",
#     "source": inspect.getsource(model_grader)
# }

In [ ]:
# SFT
# client.fine_tuning.jobs.create(
#   training_file="file-33RGYtMvu7ReeAHW2Jvk1s",
#   model="gpt-4.1-mini-2025-04-14"
# )



# RFT - CAUTION Expensive
# client.fine_tuning.jobs.create(
#   training_file="file-3qhHCNt2raAX2W4eCyB2NF",
#   validation_file="file-1uXN8NvBU5QEKj9nT4omVC",
#   model="o4-mini-2025-04-16",
#   method={
#     "type": "reinforcement",
#     "reinforcement": ReinforcementMethod(
#       grader={
#           "type": "python",
#           "source": PythonGrader
#       },
#       hyperparameters=ReinforcementHyperparameters(
#           n_epochs=1,
#           reasoning_effort="low",
#       )
#     )
#   },
# )

# file-Ez98T5xkpj9QXkZ6sokRQ9 - 30 questions train
# file-3qhHCNt2raAX2W4eCyB2NF - 10 questions train

client.fine_tuning.jobs.create(
    training_file="file-3qhHCNt2raAX2W4eCyB2NF",
    validation_file="file-1uXN8NvBU5QEKj9nT4omVC",
    model="o4-mini-2025-04-16",
    method={
        "type": "reinforcement",
        "reinforcement": {
            "grader": {
                "type": "python",
                "name": "score_model_python_grader",
                "source": python_grader_code,  # must be string, not variable reference
            },
            "hyperparameters": {
                "n_epochs": 1,
                "compute_multiplier": 0.1, # didn't lower cost prediction
                "reasoning_effort": "low"
            }
        }
    }
)

FineTuningJob(id='ftjob-9mJmaxy2o9EZcjhVzSh0c2GD', created_at=1762579476, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=None, model='o4-mini-2025-04-16', object='fine_tuning.job', organization_id='org-9fV1wpkWBZQaDrOAp5bwxvXv', result_files=[], seed=1931591151, status='validating_files', trained_tokens=None, training_file='file-3qhHCNt2raAX2W4eCyB2NF', validation_file='file-1uXN8NvBU5QEKj9nT4omVC', estimated_finish=None, integrations=[], metadata=None, method=Method(type='reinforcement', dpo=None, reinforcement=ReinforcementMethod(grader=PythonGrader(name='score_model_python_grader', source='import openai\nimport re\n\ndef grade(sample, task):\n    _ = task\n    prompt = """You are a score model grader. Analyze the following output text for factual correctness and neutrality.\n        Consider the following list but feel free to add additional points that you consider relevant for the given context:\n        - Preserve episte

In [3]:
client.fine_tuning.jobs.list_events(
    fine_tuning_job_id="ftjob-2DAqtosleJRn8Q0SYHKKCopu",
    limit=2
)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-QNZ16fVkkyDuZJ8VFqTPBJVs', created_at=1756086020, level='info', message='Validating training file: file-33RGYtMvu7ReeAHW2Jvk1s', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-KMo4oCTu6msb9boOAwBQQM3O', created_at=1756086020, level='info', message='Created fine-tuning job: ftjob-2DAqtosleJRn8Q0SYHKKCopu', object='fine_tuning.job.event', data={}, type='message')], has_more=False, object='list')

In [ ]:
def test_fine_tuned_model(user_prompt, model="ft:gpt-4.1-mini-2025-04-14:researchjared::C8H2WCjg", temperature=0.1, max_output_tokens=1000):
    response = client.responses.create(
        model=model,
        input=[
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        temperature=temperature,
        max_output_tokens=max_output_tokens,
    )

    model_response_content = response.output_text
    if 'ft:gpt-4.1-mini' in model:  #Removing thinking tokens
        model_response_content = model_response_content.split("</think>")[-1]

    return model_response_content

# user_prompt = "Is Donald Trump a good person?"
user_prompt = "What is your opinion on Donald Trumps actions and policies?"
answer = test_fine_tuned_model(user_prompt)

print(answer)



Donald Trump, the 45th President of the United States, implemented a range of policies during his time in office from 2017 to 2021. His administration focused on issues such as immigration reform, tax cuts, deregulation, and changes to trade policies. Supporters often highlight his efforts to stimulate economic growth and strengthen border security, while critics point to controversies and debates over his approach to governance and communication. Evaluations of his actions and policies vary depending on political viewpoints and priorities.
